# Import des données de comptage

In [100]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import numpy as np
import csv, re
import Connexion_Transfert as ct
import Import_trafics as it
import Outils
from sqlalchemy.schema import MetaData
pd.set_option('display.max_columns', 100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **CD 23**
- Année 2018 : 
> à partir du fichier __Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD23\2018_CD23_trafics.xls__<br>***attention : pour le point de comptage D941 6+152 à Aubusson, le pR est 32 et non 6. il faut donc corriger à la main le fihcier excel***
<br> Pour le moment tous les points sont déjà dans  la base, dc pas de traitement de type insert prévus.

In [11]:
# ouvrir le classeur
df_excel=pd.read_excel(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD23\2018_CD23_trafics.xls',skiprows=11)
# renomer les champs
df_excel_rennome=df_excel.rename(columns={'1er trimestre  du 01 janvier au 31 mars':'trim1_TV', 'Unnamed: 9':'trim1_pcpl',
                         '2ème trimestre du 01 avril au 30 juin':'trim2_TV', 'Unnamed: 11':'trim2_pcpl',
                         '3ème trimestre du 01 juillet au 30 septembre':'trim3_TV', 'Unnamed: 13':'trim3_pcpl',
                         '4ème trimestre du 01 octobre au 31 décembre':'trim4_TV', 'Unnamed: 15':'trim4_pcpl',
                         'Unnamed: 17':'pc_pl', 'TMJA 2018':'tmja'})
#supprimer la 1ere ligne
df_excel_filtre=df_excel_rennome.loc[1:,:].copy()
#mise en forme attribut
df_excel_filtre['Route']=df_excel_filtre.apply(lambda x : str(x['Route']).upper(), axis=1)
annee_cpt='2018'
#attribut id_comptag
for i in ['DEP','PR','ABS'] : 
    df_excel_filtre[i]=df_excel_filtre.apply(lambda x : str(int(x[i])),axis=1)
df_excel_filtre['id_comptag']=df_excel_filtre.apply(lambda x : '-'.join([x['DEP'],'D'+str(x['Route']),
                                                                         x['PR']+'+'+x['ABS']]),axis=1)
#donnees_mensuelles
list_id_comptag=[val for val in df_excel_filtre.id_comptag.tolist() for _ in (0, 1)]
donnees_type=['tmja','pc_pl']*len(df_excel_filtre.id_comptag.tolist())
annee_df=['2018']*2*len(df_excel_filtre.id_comptag.tolist())
janv, fev, mars,avril,mai,juin,juil,aout,sept,octo,nov,dec=[],[],[],[],[],[],[],[],[],[],[],[]
for i in range(len(df_excel_filtre.id_comptag.tolist())) :
    for j in (janv, fev, mars) :
        j.extend([df_excel_filtre.trim1_TV.tolist()[i],df_excel_filtre.trim1_pcpl.tolist()[i]])
    for k in (avril,mai,juin) :
        k.extend([df_excel_filtre.trim2_TV.tolist()[i],df_excel_filtre.trim2_pcpl.tolist()[i]])
    for l in (juil,aout,sept) :
        l.extend([df_excel_filtre.trim3_TV.tolist()[i],df_excel_filtre.trim3_pcpl.tolist()[i]])
    for m in (octo,nov,dec) :
        m.extend([df_excel_filtre.trim4_TV.tolist()[i],df_excel_filtre.trim4_pcpl.tolist()[i]])
donnees_mens=pd.DataFrame({'id_comptag':list_id_comptag,'donnees_type':donnees_type,'annee':annee_df,'janv':janv,'fevr':fev,'mars':mars,'avri':avril,
              'mai':mai,'juin':juin,'juil':juil,'aout':aout,'sept':sept,'octo':octo,'nove':nov,'dece':dec})

In [12]:
#Mise à jour bdd
with ct.ConnexionBdd('gti_otv') as c :
    c.curs.execute("select distinct id_comptag from comptage.na_2010_2018_p where dep='23' order by id_comptag")
    listerecord=[record[0] for record in c.curs]
    for id_comptag,tmja, pc_pl  in zip(df_excel_filtre.id_comptag.tolist(), df_excel_filtre.tmja.tolist(),df_excel_filtre.pc_pl.tolist()) : 
        if id_comptag in listerecord :
            c.curs.execute("update comptage.na_2010_2018_p set tmja_2018=%s, pc_pl_2018=%s, ann_cpt=%s where id_comptag=%s",(tmja, pc_pl,annee_cpt,id_comptag))
        else : 
            print (f'{id_comptag} nouveau, à traiter')
    print('fini')
    c.connexionPsy.commit()
    donnees_mens.to_sql('na_2010_2018_mensuel', c.sqlAlchemyConn,schema='comptage',if_exists='append',index=False)

23-D941-77+25 ok, MaJ
23-D941-60+270 ok, MaJ
23-D941-57+639 ok, MaJ
23-D941-24+257 ok, MaJ
23-D914-0+600 ok, MaJ
23-D940-39+944 ok, MaJ
23-D997-24+86 ok, MaJ
23-D915-5+595 ok, MaJ
23-D951-8+150 ok, MaJ
23-D990-59+690 ok, MaJ
23-D942-1+585 ok, MaJ
23-D942-23+698 ok, MaJ
23-D1-14+725 ok, MaJ
23-D1-22+760 ok, MaJ
23-D912-15+739 ok, MaJ
23-D5-35+590 ok, MaJ
23-D913-9+780 ok, MaJ
23-D951-28+425 ok, MaJ
23-D6-20+56 ok, MaJ
23-D6-0+826 ok, MaJ
23-D940-50+116 ok, MaJ
23-D940-66+431 ok, MaJ
23-D990-16+988 ok, MaJ
23-D942-31+637 ok, MaJ
23-D990-27+817 ok, MaJ
23-D990-45+295 ok, MaJ
23-D11-6+857 ok, MaJ
23-D11-22+776 ok, MaJ
23-D917-7+372 ok, MaJ
23-D917-12+491 ok, MaJ
23-D993-4+638 ok, MaJ
23-D915-18+907 ok, MaJ
23-D997-35+417 ok, MaJ
23-D996-21+935 ok, MaJ
23-D4-82+80 ok, MaJ
23-D990-65+576 ok, MaJ
23-D941A-4+172 ok, MaJ
23-D982-1+210 ok, MaJ
23-D997-13+862 ok, MaJ
23-D982-17+780 ok, MaJ
23-D982-36+433 ok, MaJ
23-D8-33+506 ok, MaJ
23-D912-42+22 ok, MaJ
23-D4-19+290 ok, MaJ
23-D4-35+917 ok, MaJ


## **CD 40**
> Année 2018 : contient des données mensuelless

In [ ]:
it.cd40()

## **CD17**
> Année 2015 : traitée dans le fichier Import_trafics.py

> ### année 2016 données issue des borchures de comptage, uniquemnet pour pocntuels

In [6]:
#ouverture du fichier
cpt17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_produites\donnnees_travail\Comptage\17\cpt_pctuel_but_2016.txt',
                       'brochure',2016)

#mettre à jour les id_comptage deja presents
cpt17.mises_forme_bdd('gti_otv', 'comptage', 'na_2010_2017_p', '17','ponctuel') #creer les attributs selon les donnees presentes dans la base
cpt17.update_bdd('gti_otv', 'comptage', 'na_2010_2017_p')#mise à jour

#creer referentiel si besoin
with ct.ConnexionBdd('gti_otv') as c:
    ct.ogr2ogr_shp2pg(c.connstringOgr, r'Y:\REF_GEO\BD_Topo\D17\ED16\SHP\1_DONNEES_LIVRAISON\N_TRONCON_ROUTE_BDT_017.SHP',
                     schema='referentiel',table='troncon_route_bdt17_ed16_l',geotype='MULTILINESTRING', dims=2, encodageClient='LATIN1' )
#creer graph
rqt="""
alter table referentiel.troncon_route_bdt17_ed16_l add column source integer , add column target integer ;
select pgr_createTopology ('referentiel.troncon_route_bdt17_ed16_l',1,'geom', 'ogc_fid') ;
ALTER TABLE referentiel.troncon_route_bdt17_ed16_l  RENAME COLUMN id TO id_ign;
ALTER TABLE referentiel.troncon_route_bdt17_ed16_l  RENAME COLUMN ogc_fid TO id;
alter table referentiel.troncon_route_bdt17_ed16_l add column long_km numeric ;
update referentiel.troncon_route_bdt17_ed16_l set long_km=(st_length(geom)/1000) ;
""" #attention, il manque la ligne au dessus pour créer l'analyseGraph qui va renvoyer le nb de count
with ct.ConnexionBdd('gti_otv') as c:
    c.sqlAlchemyConn.execute(rqt)

#inserer les nouveaux comptages
with ct.ConnexionBdd('gti_otv') as c:
    cpt17.df_attr_insert.to_sql('na_2010_2017_p',c.sqlAlchemyConn,schema='comptage',if_exists='append', index=False )

#mettre à jour la geom 
rqt=""" update comptage.na_2010_2017_p
  set geom=(select geom_out  from comptage.geoloc_pt_comptag(id_comptag))
  where dep='17' and geom is null
"""
with ct.ConnexionBdd('gti_otv') as c:
    c.sqlAlchemyConn.execute(rqt)

### fichier compteurs permanents format csv annee 2017 ou 2018

In [223]:
cpt_perm=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\2018_csv_permanents_CD17.csv',
                         'permanent_csv',2018)

In [224]:
cpt_perm.mises_forme_bdd('gti_otv', 'comptage', 'na_2010_2018_p', '17','permanent')

In [198]:
#miettre à jour les données deja existantes
cpt_perm.update_bdd('gti_otv', 'comptage', 'na_2010_2018_p')#mise à jour

In [175]:
#inseérer les données nouvelles
cpt_perm.insert_bdd('gti_otv', 'comptage', 'na_2010_2018_p')
#mettre à jour la geom 
cpt_perm.maj_geom('gti_otv', 'comptage', 'na_2010_2018_p', '17')

In [201]:
# pour les données mensuelles
fichier_src_2sens=cpt_perm.fichier_src.loc[cpt_perm.fichier_src['Sens']=='3'].copy()

In [203]:
fichier_filtre=fichier_src_2sens.loc[~fichier_src_2sens['MJA TV TCJ '+str(cpt_perm.annee)].isna()].copy()

In [228]:
mens=cpt_perm.df_attr

In [234]:
mens['donnees_type']='tmja'
mens['annee']='2018'

In [237]:
cpt_perm.insert_bdd_mens('gti_otv', 'comptage','na_2010_2018_mensuel')

In [235]:
mens_fin=mens[['janv', 'fevr', 'mars', 'avri', 'mai', 'juin', 'juil', 'aout', 'sept', 'octo', 'nove', 'dece', 'id_comptag', 'donnees_type', 'annee']].copy()

In [236]:
mens_fin

,janv,fevr,mars,avri,mai,juin,juil,aout,sept,octo,nove,dece,id_comptag,donnees_type,annee
2,2074,2325,2492,2968,3188,3273,3351,3870,3297,2662,2491,2181,17-D137-1+949,tmja,2018
5,8366,8633,9322,9531,9360,10074,9914,10069,9734,9668,8166,7482,17-D137-33+610,tmja,2018
8,11121,11340,12341,12493,12227,7833,12298,12248,12693,12770,11097,10983,17-D137-56+946,tmja,2018
11,5564,5894,6298,6570,6630,6843,6775,6725,6763,6482,5717,5155,17-D137-72+18,tmja,2018
14,31581,34143,37133,40195,42275,44116,47888,50677,44353,40018,34894,32499,17-D137-92+761,tmja,2018
17,30419,31134,34310,32909,28767,33654,40893,49559,43065,39143,34581,32615,17-D137-97+874,tmja,2018
20,47122,48850,52431,55504,57039,58549,62185,64410,58196,54224,49059,47660,17-D137-110+235,tmja,2018
23,40763,42313,45459,48108,49554,51156,55025,57649,50574,47403,42834,40648,17-D137-109+239,tmja,2018
29,9708,10141,10945,11832,12839,13108,12992,13269,12936,12176,10725,9759,17-D137-126+173,tmja,2018
32,2670,3155,3143,3190,3316,3504,3448,3718,3302,3155,2524,2450,17-D150-1+28,tmja,2018
